In [3]:
import numpy as np 

In [4]:
class ActivationFunction: 
    def __init__(self, func, derivative): 
        self.func = func
        self.derivative = derivative

    def __call__(self, x): 
        return self.func(x) 
    
class LossFunction(ActivationFunction): 
    def __call__(self, y, y_hat): 
        return self.func(y, y_hat)

In [68]:
def _sigmoid(x): 
    return 1 / (1 + np.exp(-x))

sigmoid = ActivationFunction(
    func=lambda x: _sigmoid(x),
    derivative=lambda x: _sigmoid(x) * (1 - _sigmoid(x))
)
relu = ActivationFunction(
    func=lambda x: np.maximum(0, x),
    derivative=lambda x: np.where(x > 0, 1, 0)
)
tanh = ActivationFunction(
    func=lambda x: np.tanh(x),
    derivative=lambda x: 1 - np.tanh(x) ** 2
)
mse = LossFunction(
    func=lambda y_true, y_pred: np.mean((y_true - y_pred) ** 2), # The function is the average of the squared differences
    derivative=lambda y_true, y_pred: 2 * (y_pred - y_true) / y_true.size # The derivative is the negative of the average of the differences times 2
)

In [69]:
class Layer:
    def __init__(self, n_input, n_output, activation):
        self.n_input = n_input # The number of input nodes
        self.n_output = n_output # The number of output nodes
        self.activation = activation # The activation function
        self.W = np.random.randn(n_input, n_output) # The weight matrix
        self.b = np.random.randn(n_output) # The bias vector
        self.Z = None # The input to the layer
        self.A = None # The output of the layer
        self.E = None # The error of the layer
        self.dZ = None # The derivative of the error of the layer

    def forward(self, X):
        self.Z = X.dot(self.W) + self.b # Compute the input to the layer
        self.A = self.activation(self.Z) # Compute the output of the layer
        return self.A # Return the output

    def backward(self, E):
        self.E = E # Assign the propagated error to the layer's error
        self.dZ = self.E * self.activation.derivative(self.Z) # Compute the derivative of the error of the layer and assign it to self.dZ
        return self.dZ # Return the derivative

    def update(self, X, alpha):
        self.W -= alpha * np.dot(X.T, self.dZ) # Update the weight matrix
        self.b -= alpha * np.sum(self.dZ, axis=0) # Update the bias vector


In [81]:
class Network:
    def __init__(self, layers, loss):
        self.layers = layers # The list of layers
        self.loss = loss # The loss function

    def forward(self, X):
        A = X # Initialize the output as the input
        for layer in self.layers: # For each layer in the network
            A = layer.forward(A) # Forward propagate through the layer and update the output
        return A # Return the final output

    def backward(self, y):
        E = self.loss.derivative(y, self.layers[-1].A) # Compute the error at the output layer using the loss function derivative
        for i in reversed(range(len(self.layers))): # For each layer in reverse order
            E = self.layers[i].backward(E) # Backward propagate through the layer and update the error
            if i != 0: # Except for the input layer
                E = np.dot(E, self.layers[i].W.T) # propagate the error to the next layer (in reverse order)

    def update(self, X, alpha):
        for layer in self.layers: # For each layer in the network
            layer.update(X, alpha) # Update the weights and biases of the layer using the output of the previous layer
            X = layer.A # Update the input for the next layer

    def train(self, X, y, alpha, epochs):
        for epoch in range(epochs): # For each epoch
            A = self.forward(X) # Forward propagate through the network and get the output
            E = self.loss(y, A) # Compute the error at the output using the loss function
            self.backward(y) # Backward propagate through the network and update the derivatives
            self.update(X, alpha) # Update the weights and biases of the network
            if epoch % 10000 == 0: # Print the error every 1000 epochs
                print(f"Epoch {epoch}, Error: {np.mean(np.abs(E))}")

In [98]:
# Define a simple XOR network with two hidden layers and sigmoid activation function
net = Network(
    layers=[
        Layer(n_input=2, n_output=12, activation=relu),
        Layer(n_input=12, n_output=24, activation=relu),
        Layer(n_input=24, n_output=24, activation=relu),
        Layer(n_input=24, n_output=12, activation=relu), 
        Layer(n_input=12, n_output=1, activation=relu)
    ],
    loss=mse # Use the mean squared error loss function
)

# Generate random X values between 0 and 10
X = np.random.uniform(0, 10, (1000, 2))

# Calculate corresponding y values
y = 2*X[:, 0] + 3*X[:, 1]

# Reshape y to be a column vector
y = y.reshape(-1, 1)# Define the input and output data

# Train the network using backpropagation
net.train(X, y, alpha=0.00001, epochs=100000)

# Test the network with the same input data
print("Output after training:")
print(net.forward(X))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2964425942.py, line 6)